In [ ]:
import pandas as pd
import numpy as np
import logging
logging.basicConfig(level=logging.INFO)

Dataset was gathered using the code from the second task,and saved in this directory.

In [ ]:
dataset = pd.read_pickle('dataset_academy.pickle')

In [ ]:
logging.info(f"Dataset length :{len(dataset)}")
logging.info(f"Dataset columns :{dataset.columns}")

In [ ]:
dataset.reset_index(drop=True, inplace=True)
dataset.head()

In [ ]:
dataset['campaign'].value_counts()

In [ ]:
for columns in dataset.columns:
   logging.info(f"Number if missing values in {columns} : {dataset[columns].isnull().sum()}")

In [ ]:
logging.info(f"Different countries in the dataset : {dataset['geo_country'].nunique()}")

In [ ]:
logging.info(f"Different campaigns represented in the dataset : {dataset['campaign'].nunique()}")

In [ ]:
# define classes: class 1 are organic users and class 2 are campaign users
dataset['user_class'] = np.where(dataset['campaign'] == '<organic>', '1', '2')


In [ ]:
# for column in dataset.columns:
#    if dataset[column].dtype == 'float64':
#       if np.mean(dataset[column], axis=0) <= 0.1:
#        logging.info(f"Column {column} ")

Initial thoughts on the model:

- use semi-supervised learning, fit a model on non organic data and through n iterations test model on organic data, all organic data with high probability of being organic will be added to the training set,
- use probabilistic method; starting with certainty of 1 for non organic users to be non organic and soft label of 0.5/0.7 for organic users to be organic, build classifier and update probabilities of organic users, if they drop belove certain tresold, they are added to the non organic users
- use ensamble methods, train multiple models on non organic data and use them to classify organic data, if they mostly agree on the classification, add the user to the non organic users

Comments
- I will probably not add synthetic data, as I lack domain knowledge to be certain in my data ingestion.

In [ ]:
dataset.drop(columns=['user_id', 'campaign'], inplace=True)

In [ ]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse_output=False)

In [ ]:
dataset['geo_country']= encoder.fit_transform(dataset['geo_country'].reshape(-1, 1))
dataset

In [ ]:
organic_users,non_organic_users = dataset[dataset['user_class'] == '1'],dataset[dataset['user_class'] == '2']
organic_users.reset_index(drop=True, inplace=True)
non_organic_users.reset_index(drop=True, inplace=True)

In [ ]:
logging.info(f"Number of organic users : {len(organic_users)}")
logging.info(f"Number of non-organic users : {len(non_organic_users)}")

Modeling

Semi-supervised learning

Random Forests

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [ ]:
X = non_organic_users.drop(columns=['user_class'])
y = non_organic_users['user_class']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [ ]:
randomForestClassifier = RandomForestClassifier(n_estimators=1000, random_state=42)

In [ ]:
randomForestClassifier.fit(X_train, y_train)